## mixQTL inputs for GTEx v8 data

This notebook describes the pre-processing steps required for running the mixQTL model in tensorQTL using data fr om the GTEx v8 release:
1. Converting the phased VCF to haplotype dosages, stored as two separate Parquet files.
2. Converting the per-individual ASE files to per-tissue summaries.

The input files can be retrieved from [AnVIL](https://anvil.terra.bio/#workspaces/anvil-datastorage/AnVIL_GTEx_V8_hg38) (required dbGaP access). The ASE files are in [GTEx_Analysis_2017-06-05_v8_ASE_WASP_counts_by_subject](gs://fc-secure-ff8156a3-ddf3-42e4-9211-0fd89da62108/GTEx_Analysis_2017-06-05_v8_ASE_WASP_counts_by_subject/) and the VCF in [GTEx_Analysis_2017-06-05_v8_WGS_VCF_files](gs://fc-secure-ff8156a3-ddf3-42e4-9211-0fd89da62108/GTEx_Analysis_2017-06-05_v8_WGS_VCF_files/).

The GTF file `gencode.v26.GRCh38.genes.gtf` required for remapping variants to genes in the ASE files is available [here](https://console.cloud.google.com/storage/browser/_details/gtex-resources/GENCODE/gencode.v26.GRCh38.genes.gtf) and from the [GTEx Portal](https://gtexportal.org/home/datasets).


In [ ]:
import pandas as pd
import numpy as np
import gzip
import os
import subprocess
import glob
from collections import defaultdict
from bx.intervals.intersection import IntervalTree
import qtl.annotation as annotation


### Convert VCF to haplotype dosages

In [ ]:
cmd = 'python3 ../utils/vcf2hap.py \
    GTEx_Analysis_2017-06-05_v8_WholeGenomeSeq_838Indiv_Analysis_Freeze.SHAPEIT2_phased.MAF01.vcf.gz \
    --output_dir .'
subprocess.check_call(cmd, shell=True)


In [ ]:
# the dosages can then be loaded as
hap1_df = pd.read_parquet('GTEx_Analysis_2017-06-05_v8_WholeGenomeSeq_838Indiv_Analysis_Freeze.SHAPEIT2_phased.MAF01.hap1.parquet')
hap2_df = pd.read_parquet('GTEx_Analysis_2017-06-05_v8_WholeGenomeSeq_838Indiv_Analysis_Freeze.SHAPEIT2_phased.MAF01.hap2.parquet')


### Aggregate ASE reads by tissue and gene

In [ ]:
# 1) convert ASE tables to parquet for faster processing
ase_files = {os.path.basename(i).split('.')[0]:i for i in glob.glob('GTEx_Analysis_v8_ASE_WASP_counts_by_subject/*.v8.wasp_corrected.ase_table.tsv.gz')}
for k, donor_id in enumerate(sorted(ase_files), 1):
    print('\rProcessing participant {}/{}'.format(k, len(ase_files)), end='')
    df = pd.read_csv(ase_files[donor_id], sep='\t')
    df.to_parquet('GTEx_Analysis_v8_ASE_WASP_counts_by_subject_parquet/{}.v8.wasp_corrected.ase_table.parquet'.format(donor_id))


In [ ]:
def aggregate_ase(ase_file_dict, tissue_abbrv, gene_interval_trees, filter_flagged=True):
    """Aggregate ASE counts per gene, and generate table for the selected tissue"""
    cumul_df = []
    for k, donor_id in enumerate(sorted(ase_files), 1):
        print('\rProcessing participant {}/{}'.format(k, len(ase_files)), end='')

        if ase_files[donor_id].endswith('.parquet'):
            df = pd.read_parquet(ase_files[donor_id])
        else:
            df = pd.read_csv(ase_files[donor_id], sep='\t')

        df = df[df['TISSUE_ID']==tissue_abbrv]

        if filter_flagged:
            df = df[(df['LOW_MAPABILITY']==0) & 
                    (df['MAPPING_BIAS_SIM']==0) & 
                    (df['GENOTYPE_WARNING']==0)]

        # gene IDs in ASE files are from VEP, remap using 
        # collapsed gene model instead, so that they match eQTLs
        df.rename(columns={'GENE_ID':'GENE_ID_VEP'}, inplace=True)
        
        gene_ids = []
        for chrom, pos in zip(df['CHR'], df['POS']):
            cand = gene_interval_trees[chrom].find(pos, pos)
            if len(cand) == 0:
                gene_ids.append(np.NaN)
            elif len(cand) == 1:
                gene_ids.append(cand[0].id)
            else:  # should never happen with collapsed model
                raise ValueError('Multiple matches')
        df['GENE_ID'] = gene_ids
        df = df[df['GENE_ID'].notnull()]
        df = df.sort_values(['TISSUE_ID', 'GENE_ID'])

        if df.shape[0] > 0:
            ref_cumul = defaultdict(int)
            alt_cumul = defaultdict(int)
            for gene_id, ref, alt in zip(df['GENE_ID'], df['REF_COUNT'], df['ALT_COUNT']):
                ref_cumul[gene_id] += ref
                alt_cumul[gene_id] += alt
            cdf = pd.concat([pd.Series(ref_cumul, name='ref'), pd.Series(alt_cumul, name='alt')], axis=1)
            cdf.index.name = 'gene_id'
            cdf['donor_id'] = donor_id
            cumul_df.append(cdf)
    cumul_df = pd.concat(cumul_df, axis=0).reset_index()
    cumul_df = cumul_df.sort_values(['gene_id', 'donor_id']).reset_index(drop=True)
    return cumul_df


annot = annotation.Annotation('gencode.v26.GRCh38.genes.gtf', verbose=False)
gene_interval_trees = defaultdict()
for g in annot.genes:
    for e in g.transcripts[0].exons:
        gene_interval_trees.setdefault(g.chr, IntervalTree()).add(e.start_pos, e.end_pos+1, g)

ase_files = {os.path.basename(i).split('.')[0]:i for i in glob.glob('GTEx_Analysis_v8_ASE_WASP_counts_by_subject_parquet/*.v8.wasp_corrected.ase_table.parquet')}
ase_df = aggregate_ase(ase_files, 'WHLBLD', gene_interval_trees, filter_flagged=True)
ase_df.to_parquet('Whole_Blood.v8.wasp_corrected.ase_gene.parquet')
